In [2]:
# Just take all width for viz
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

import os

In [ ]:
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config("spark.driver.memory", "10g") \
    .appName("steam-analysis-eda") \
    .getOrCreate()

### Part 2 : Games Daily : usage du joueur

- Game playing data for a select subset of users. Each user's data in the subset was requested repeatedly, every day for five days.

#### Combien de jeux un joueur possède-t-il ?

Comme nous le verrons à l'étape de visualisation, la distribution du nombre de jeux possédés est complètement asymétrique. Il faudra donc utiliser une médiane.

Le jeu de donnée qui suit n'a pas été filtré par éditeur / développeur ! Les types n'ont pas non plus été filtré. Il faudra appliquer à nouveaux les filtres.

In [ ]:
df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Daily")

df.printSchema()

df = df \
    .groupBy('steamid') \
    .agg(F.collect_list('appid').alias('owned_appid')) \
    .withColumn('n_owned', F.size(F.col('owned_appid'))) \
    .sort(F.col('n_owned'))

df.printSchema()

n_owned_median = df \
    .approxQuantile('n_owned', [0.5], 0.01)[0]

print('Number of owned app median :', n_owned_median)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x = df.toPandas().n_owned,
        name = "N Owned"
    )
)

fig.update_layout(
    title = 'Number of owned apps histogram',
    template = 'plotly_dark'
)

fig.show()

In [ ]:
# Filter extreme n_owned values
max_owned = 500

filtered_df = df \
    .filter(F.col('n_owned') < max_owned) \
    .sort(F.col('n_owned').desc())

# Display filtered results
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x = filtered_df.toPandas().n_owned,
        name = "N Owned"
    )
)

fig.update_layout(
    title = 'Number of owned apps histogram',
    template = 'plotly_dark'
)

fig.show()

- D'après la forme très irrégulière de l'histograme même après filtrage des valeurs extrêmes, le dataset est probablement encore très bruité.
- Comme indiqué, il faut appliquer les filtres utilisés plus haut (Type="game" et Developer != "Valve") pour obtenir une bonne vue générale.

#### Filtrer les appid pour ne garder que les jeux non-Valve

In [ ]:
df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Daily")

# Using previously cleaned dataset, without any Valve appid to implicitly remove appid with inner join
# Previous dataset also filter non-game appid
publishers = spark.sql(
    'select appid from parquet.`{}`'.format("file://" + output_dir + "steam-dataset_games_28-12_3")
)

df = df \
    .join(publishers, on = 'appid') \
    .dropna()

df = df \
    .groupBy('steamid') \
    .agg(F.collect_set('appid').alias('owned_appid')) \
    .withColumn('n_owned', F.size(F.col('owned_appid'))) \
    .sort(F.col('n_owned').desc())

df.show()

In [ ]:
import plotly.graph_objects as go

# Filter extreme n_owned values
max_owned = 500

filtered_df = df \
    .filter(F.col('n_owned') <= max_owned) \
    .sort(F.col('n_owned'))

# Get some describe statistics
filtered_df.describe('n_owned').show()

# Display filtered results
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x = filtered_df.toPandas().n_owned,
        name = "N Owned"
    )
)

fig.update_layout(
    title = 'Number of owned game histogram',
    template = 'plotly_dark'
)

fig.show()

#### Combinaisons de genres

En regardant les possessions des joueurs, on peut s'attendre à retrouver des genres hybrides connus en triant les résultats par co-occurences.

- Chargement du dataset des genres et récupération de tous les genres distincts se retrouvant dans les appid possédées.

In [ ]:
genre_df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Genres")
genre_old_df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Genres_Old")

genre_df = genre_df.union(genre_old_df)

df = df \
    .select('steamid', F.explode('owned_appid').alias('appid')) \
    .join(games_df, on = 'appid') \
    .join(genre_df, on = 'appid') \
    .groupBy('steamid') \
    .agg(F.collect_set('Genre').alias('game_genres')) \
    .join(df, on = 'steamid')

df.show()

#### Aggrégation

Cette étape consiste à utiliser la colonne nouvellement crée pour compter les co-occurences de genres.

Pour obtenir ces valeurs, la tâche sera découpée en deux partie qui illustrent parfaitement un job MapReduce simple :

  - Map : combinaison (par 2) des différents genres possédés par un joueur. A cette étape, pour chaque joueur, un dict sera créé avec la forme {"Genre1-Genre2" : 1}
  - Reduce : Une fois le mapping créé, on va effectuer une aggrgation sur les différents binômes de genres obtenus.

#### Création d'une User Defined Function pour l'étape de Map

**NOTES** : Les jeux possédés sont crés par un SET, le count n'est pas nécessaire. Il suffit de faire correspondre "1" pour chaque key.

- Amélioration : compter les multiples occurences d'un même genre hybride.
    - example : Si 2 RPG-Action possédés => {"RPG-Action" : 2}


- steps : 
    - Remplacer "collect_set" par "collect_list"
    - Récupérer pour chaque joueur les combinaisons distinctes
    - Pour chaque combinaison, compter le nombre d'occurence

In [ ]:
from itertools import combinations
from collections import Counter

def comb_count(x):
    count = Counter(list(combinations(x, 2)))
    count = dict(zip(["-".join(x) for x in count.keys()], count.values()))
    return count

example = ['RPG', 'Massively Multiplayer', 'Action', 'Indie', 'Adventure']
comb_count(example)

In [ ]:
count_udf = F.udf(comb_count, MapType(StringType(), IntegerType()))

mixed_genres_counts = df \
    .withColumn('cooccurrences', count_udf(df['game_genres'])) \
    .select(F.explode('cooccurrences')) \
    .groupBy('key') \
    .agg(F.sum('value').alias('mixed_genres_sum')) \
    .sort(F.col('mixed_genres_sum').desc())

mixed_genres_counts.show(truncate = False)

In [ ]:
mixed_genres_stats = mixed_genres_counts.select('mixed_genres_sum').describe()

mixed_genres_stats.show()

In [ ]:
# Display filtered results
fig = go.Figure()

pd_df = mixed_genres_counts \
    .toPandas()

fig.add_trace(
    go.Bar(
        x = pd_df.key,
        y = pd_df.mixed_genres_sum,
        name = "Mixed genres counts"
    )
)

fig.update_layout(
    title = 'Mixed genre games counts',
    template = 'plotly_dark'
)
fig.update_xaxes(tickangle = 45)

fig.show()

- Nettoyage du dataset pour une meilleure lisibilité directe
    - Plutôt que d'utiliser un seuil en valeur absolue, nous allons calculer cette valeur à partir du dataset pour garantir la compatibilité du seuil avec le dataset, et assurer la généralisation de la démarche. Si le dataset change, la valeur accompagne les évolutions.
    - Distribution très asymétrique : Utilisation de la médiane

In [ ]:
# Set percentile threshold
pop_threshold = 0.5

threshold = mixed_genres_counts \
    .approxQuantile('mixed_genres_sum', [pop_threshold], 0.01)[0]
    
print('Threshold:', threshold)

In [ ]:
# Display filtered results
fig = go.Figure()

pd_df = mixed_genres_counts \
    .filter(F.col('mixed_genres_sum') > threshold) \
    .toPandas()

fig.add_trace(
    go.Bar(
        x = pd_df.key,
        y = pd_df.mixed_genres_sum,
        name = "Mixed genres counts"
    )
)

fig.update_layout(
    title = 'Mixed genre games counts',
    template = 'plotly_dark'
)

fig.update_xaxes(tickangle = 45)

fig.show()

In [ ]:
fig = go.Figure()

filtered_pd_df = pd_df \
    .loc[pd_df.key.str.contains('Free to Play') == False] \
    .loc[pd_df.key.str.contains('Early Access') == False]

fig.add_trace(
    go.Bar(
        x = filtered_pd_df.key,
        y = filtered_pd_df.mixed_genres_sum,
        name = "Mixed genres counts"
    )
)

fig.update_layout(
    title = 'Mixed genre games counts',
    template = 'plotly_dark'
)

fig.update_xaxes(tickangle = 45)

fig.show()

- distance entre RPG-Massively Multiplayer / Massively Multiplayer - Early Access
- Trouver quel genre comporte le plus d'early access
    - prédire l'évolution de l'early access par genre en fonction du temps
- TO-DO : retirer "Free to Play" et "Early Access" avant visualisation ! Les valeurs doivent être retirées AVANT la création des combinaisons.

Notes : quand fait avec Pandas, pas d'association "Massively Multiplayer-RPG"

In [ ]:
pd_df \
    .loc[pd_df.key.str.contains('Free to Play') == False] \
    .loc[pd_df.key.str.contains('Early Access') == False] \
    .loc[pd_df.key.str.contains('Massively Multiplayer')]

**NOTES** :

- Créer un network à partir des co-occurences avec col_0 = src, col_1 = dst, col_2 = weight
- Visualisation du network
- Visualiser les profils de joueurs ET les counts de genres hybrides en normalisant les deux scores entre 0 et 1 (TEST)
- Déplacer l'analyse du profil du joueur dans un autre chapitre, et créer plus de visualisations sur les données consolidées dans la Part 1